# Walmart : predict weekly sales


    Part 1 : make an EDA and all the necessary preprocessings to prepare data for machine learning
    Part 2 : train a linear regression model (baseline)
    Part 3 : avoid overfitting by training a regularized regression model


In [2]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.figure_factory as ff


from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.impute import SimpleImputer, KNNImputer
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.metrics import r2_score, root_mean_squared_error
from sklearn.model_selection import cross_val_score



## Part 1 : EDA

- The dataset references the weekly sales of 20 stores. 
- Each of the 150 entries represents the weekly sales of a store, recorded at the end of the week (Friday).
- The sales data ranges from February 5, 2010, to October 19, 2012.
- The dataset aslo includes contextual information such as :
    - Holiday events occuring during the week
    - Recorded temperatures
    - Recorded fuel prices
    - The Consumer Price Index, which measures the annual inflation rate
    - The unempoyment rate


In [3]:
df= pd.read_csv("data/Walmart_Store_sales.csv")
print(df.shape)
df.head()

(150, 8)


,Store,Date,Weekly_Sales,Holiday_Flag,Temperature,Fuel_Price,CPI,Unemployment
0,6.0,18-02-2011,1572117.54,NaN,59.61,3.045,214.777523,6.858
1,13.0,25-03-2011,1807545.43,0.0,42.38,3.435,128.616064,7.470
2,17.0,27-07-2012,NaN,0.0,NaN,NaN,130.719581,5.936
3,11.0,NaN,1244390.03,0.0,84.57,NaN,214.556497,7.346
4,6.0,28-05-2010,1644470.66,0.0,78.89,2.759,212.412888,7.092


In [4]:
df.describe(include= "all")

,Store,Date,Weekly_Sales,Holiday_Flag,Temperature,Fuel_Price,CPI,Unemployment
count,150.000000,132,1.360000e+02,138.000000,132.000000,136.000000,138.000000,135.000000
unique,NaN,85,NaN,NaN,NaN,NaN,NaN,NaN
top,NaN,19-10-2012,NaN,NaN,NaN,NaN,NaN,NaN
freq,NaN,4,NaN,NaN,NaN,NaN,NaN,NaN
mean,9.866667,NaN,1.249536e+06,0.079710,61.398106,3.320853,179.898509,7.598430
std,6.231191,NaN,6.474630e+05,0.271831,18.378901,0.478149,40.274956,1.577173
min,1.000000,NaN,2.689290e+05,0.000000,18.790000,2.514000,126.111903,5.143000
25%,4.000000,NaN,6.050757e+05,0.000000,45.587500,2.852250,131.970831,6.597500
50%,9.000000,NaN,1.261424e+06,0.000000,62.985000,3.451000,197.908893,7.470000
75%,15.750000,NaN,1.806386e+06,0.000000,76.345000,3.706250,214.934616,8.150000


In [5]:
df.isnull().sum()

Store            0
Date            18
Weekly_Sales    14
Holiday_Flag    12
Temperature     18
Fuel_Price      14
CPI             12
Unemployment    15
dtype: int64

Almost all columns, including the target variable, have missing values.  

In [6]:
df1 = df.copy()

In [7]:
# As Date column cannot be included as it is in the model, let's create new columns that could contain numeric features :
df1['Date'] = pd.to_datetime(df1["Date"], format='%d-%m-%Y')
df1['Day'] = df1['Date'].dt.day
df1['Month'] = df1['Date'].dt.month
df1['Year'] = df1['Date'].dt.year
df1["Day_of_week"] = df1["Date"].dt.dayofweek
df1['Week_of_year'] = df1['Date'].dt.isocalendar().week
df1.dtypes

Store                  float64
Date            datetime64[ns]
Weekly_Sales           float64
Holiday_Flag           float64
Temperature            float64
Fuel_Price             float64
CPI                    float64
Unemployment           float64
Day                    float64
Month                  float64
Year                   float64
Day_of_week            float64
Week_of_year            UInt32
dtype: object

In [8]:
df1['Day'].value_counts().sort_values(ascending=False)

Day
12.0    11
19.0     8
18.0     7
7.0      7
25.0     7
10.0     7
30.0     7
16.0     6
26.0     6
27.0     6
6.0      5
22.0     5
20.0     5
2.0      4
24.0     4
17.0     4
13.0     4
28.0     3
11.0     3
3.0      3
9.0      3
15.0     3
4.0      2
23.0     2
1.0      2
14.0     2
5.0      2
29.0     2
31.0     1
8.0      1
Name: count, dtype: int64

In [9]:
# As there is only a small number of entry per day, let's group them per week
conditions = [
    (df1['Day'] <= 7),
    (df1['Day'] > 7) & (df1['Day'] <= 14),
    (df1['Day'] > 14) & (df1['Day'] <= 21),
    (df1['Day'] > 21) & (df1['Day'] <= 28),
    (df1['Day'] > 28)
]

# Définir les valeurs correspondantes pour chaque condition
values = [1, 2, 3, 4, 5]

# Appliquer les conditions à la colonne 'week'
df1['week'] = np.select(conditions, values, default=None).astype(float)

df1 = df1.drop("Day", axis=1)

In [10]:
df1['Day_of_week'].value_counts().sort_index()

Day_of_week
4.0    132
Name: count, dtype: int64

In [11]:
# Day_of_week having only one unique value (Friday), this variable is not interesting to keep
df1= df1.drop("Day_of_week", axis = 1)

In [12]:
df1['week'].value_counts().sort_index()

week
1.0    25
2.0    31
3.0    33
4.0    33
5.0    10
Name: count, dtype: int64

In [13]:
df1['Month'].value_counts().sort_index()

Month
1.0      7
2.0     12
3.0     11
4.0     10
5.0     15
6.0     14
7.0     15
8.0     11
9.0     10
10.0     9
11.0     8
12.0    10
Name: count, dtype: int64

In [14]:
df1['Year'].value_counts().sort_index()

Year
2010.0    54
2011.0    43
2012.0    35
Name: count, dtype: int64

In [15]:
df1['Week_of_year'].value_counts().sort_index()

Week_of_year
1     5
2     1
3     1
5     2
6     4
7     4
8     2
9     1
10    1
11    3
12    5
13    2
15    3
16    3
17    3
18    3
19    4
20    2
21    6
22    3
23    2
24    3
25    6
26    2
27    4
28    1
29    2
30    6
31    2
32    3
33    2
34    4
35    1
36    3
37    2
38    4
39    1
40    1
41    3
42    4
45    4
46    3
47    1
49    4
50    4
52    2
Name: count, dtype: Int64

There is not enouth entries per week of the year to use this variable in the model. we will need to drop it as well. but first, let's see some visualization:

In [16]:
# Calculate Total sales for each time period
Total_sales_per_date = df1.groupby('Date')['Weekly_Sales'].sum().reset_index()
Total_sales_per_week = df1.groupby('week')['Weekly_Sales'].sum().reset_index()
Total_sales_per_month = df1.groupby('Month')['Weekly_Sales'].sum().reset_index()
Total_sales_per_year = df1.groupby('Year')['Weekly_Sales'].sum().reset_index()
Total_sales_per_week_of_year = df1.groupby('Week_of_year')['Weekly_Sales'].sum().reset_index()

# Create a figure with a 3x2 grid of subplots
fig = make_subplots(rows=3, cols=2, subplot_titles=(
    'Total Weekly Sales per Date',
    'Total Weekly Sales per week',
    'Total Weekly Sales per Month',
    'Total Weekly Sales per Year',
    'Total Weekly Sales per Week_of_year'
))

# Plot : Total Weekly Sales per Date
fig.add_trace(
    px.bar(Total_sales_per_date, x='Date', y='Weekly_Sales', color_discrete_sequence=['Plum']).data[0],
    row=1, col=1
)

# Plot 1: Total Weekly Sales per week
fig.add_trace(
    px.bar(Total_sales_per_week, x='week', y='Weekly_Sales', color_discrete_sequence=['skyblue']).data[0],
    row=1, col=2
)

# Plot 2: Total Weekly Sales per Month
fig.add_trace(
    px.bar(Total_sales_per_month, x='Month', y='Weekly_Sales', color_discrete_sequence=['lightgreen']).data[0],
    row=2, col=1
)

# Plot 3: Total Weekly Sales per Year
fig.add_trace(
    px.bar(Total_sales_per_year, x='Year', y='Weekly_Sales', color_discrete_sequence=['Khaki']).data[0],
    row=2, col=2
)

# Plot 4: Total Weekly Sales per Week_of_year
fig.add_trace(
    px.bar(Total_sales_per_week_of_year, x='Week_of_year', y='Weekly_Sales', color_discrete_sequence=['salmon']).data[0],
    row=3, col=1
)

# Update layout to prevent overlap
fig.update_layout(showlegend=False, title_text="Total Weekly Sales by Different Time Periods")

# Show the plot
fig.show()


In [17]:
# Calculate average sales for each time period
average_sales_per_date = df1.groupby('Date')['Weekly_Sales'].mean().reset_index()
average_sales_per_week = df1.groupby('week')['Weekly_Sales'].mean().reset_index()
average_sales_per_month = df1.groupby('Month')['Weekly_Sales'].mean().reset_index()
average_sales_per_year = df1.groupby('Year')['Weekly_Sales'].mean().reset_index()
average_sales_per_week_of_year = df1.groupby('Week_of_year')['Weekly_Sales'].mean().reset_index()

# Create a figure with a 2x2 grid of subplots
fig = make_subplots(rows=3, cols=2, subplot_titles=(
    'Average Weekly Sales per Date',
    'Average Weekly Sales per week',
    'Average Weekly Sales per Month',
    'Average Weekly Sales per Year',
    'Average Weekly Sales per Week_of_year'
))

# Plot : Average Weekly Sales per Date
fig.add_trace(
    px.bar(average_sales_per_date, x='Date', y='Weekly_Sales', color_discrete_sequence=['Plum']).data[0],
    row=1, col=1
)

# Plot 1: Average Weekly Sales per week
fig.add_trace(
    px.bar(average_sales_per_week, x='week', y='Weekly_Sales', color_discrete_sequence=['skyblue']).data[0],
    row=1, col=2
)

# Plot 2: Average Weekly Sales per Month
fig.add_trace(
    px.bar(average_sales_per_month, x='Month', y='Weekly_Sales', color_discrete_sequence=['lightgreen']).data[0],
    row=2, col=1
)

# Plot 3: Average Weekly Sales per Year
fig.add_trace(
    px.bar(average_sales_per_year, x='Year', y='Weekly_Sales', color_discrete_sequence=['Khaki']).data[0],
    row=2, col=2
)

# Plot 4: Average Weekly Sales per Week_of_year
fig.add_trace(
    px.bar(average_sales_per_week_of_year, x='Week_of_year', y='Weekly_Sales', color_discrete_sequence=['salmon']).data[0],
    row=3, col=1
)

# Update layout to prevent overlap
fig.update_layout(showlegend=False, title_text="Average Weekly Sales by Different Time Periods")

# Show the plot
fig.show()


In [18]:
average_sales_per_store = df1.groupby('Store')['Weekly_Sales'].mean().reset_index()
total_sales_per_store = df1.groupby('Store')['Weekly_Sales'].sum().reset_index()
max_sales_per_store = df1.groupby('Store')['Weekly_Sales'].max().reset_index()
min_sales_per_store = df1.groupby('Store')['Weekly_Sales'].min().reset_index()


# Create a figure with a 1x2 grid of subplots
fig = make_subplots(rows=2, cols=2, subplot_titles=(
    'Total Weekly Sales per Store',
    'Average Weekly Sales per Store',
    'Max Weekly Sales per Store',
    'Min Weekly Sales per Store',
))

# Plot 1: Total Weekly Sales per Store
fig.add_trace(
    px.bar(total_sales_per_store, x='Store', y='Weekly_Sales', color_discrete_sequence=['lightgreen']).data[0],
    row=1, col=1
)

# Plot 2: Average Weekly Sales per Store
fig.add_trace(
    px.bar(average_sales_per_store, x='Store', y='Weekly_Sales', color_discrete_sequence=['skyblue']).data[0],
    row=1, col=2
)

# Plot 3: Max Weekly Sales per Store
fig.add_trace(
    px.bar(max_sales_per_store, x='Store', y='Weekly_Sales', color_discrete_sequence=['salmon']).data[0],
    row=2, col=1
)

# Plot 4: Min Weekly Sales per Store
fig.add_trace(
    px.bar(min_sales_per_store, x='Store', y='Weekly_Sales', color_discrete_sequence=['Khaki']).data[0],
    row=2, col=2
)
# Update layout to prevent overlap
fig.update_layout(showlegend=False, title_text="weekly Sales per store")

# Show the plot
fig.show()

In [19]:
df1["Store"].value_counts().sort_index(ascending=True)

Store
1.0     11
2.0      8
3.0     15
4.0      7
5.0      9
6.0      7
7.0      8
8.0      8
9.0      4
10.0     5
11.0     3
12.0     5
13.0     9
14.0     9
15.0     4
16.0     4
17.0     8
18.0    10
19.0     9
20.0     7
Name: count, dtype: int64

In [20]:
# List of columns to plot
columns_to_plot = ['Weekly_Sales', 'Temperature', 'Fuel_Price', 'CPI', 'Unemployment']

# Create a subplot grid
fig = make_subplots(rows=1, cols=len(columns_to_plot), subplot_titles=columns_to_plot)

# Plot a box plot and add mean marker for each column
for i, col in enumerate(columns_to_plot, start=1):
    # Box plot trace
    box_plot = px.box(df1, y=col).data[0]
    fig.add_trace(box_plot, row=1, col=i)

    # Compute and add mean marker
    mean_val = df1[col].mean()
    fig.add_trace(
        go.Scatter(
            x=[0],  # dummy x because subplot is vertical
            y=[mean_val],
            mode='markers',
            marker=dict(color='red', size=8, symbol='diamond'),
            showlegend=False,
            hovertemplate=f"Mean: {mean_val:.2f}<extra></extra>"
        ),
        row=1,
        col=i
    )

# Update layout
fig.update_layout(
    title_text="Box Plots with Mean Overlay (Red Diamond)",
    showlegend=False,
    height=500,
    width=200 * len(columns_to_plot)
)

# Show the plot
fig.show()


In [21]:
df1[df1["Unemployment"]>10]

,Store,Date,Weekly_Sales,Holiday_Flag,Temperature,Fuel_Price,CPI,Unemployment,Month,Year,Week_of_year,week
30,12.0,2011-05-06,1021154.48,0.0,68.40,4.193,129.044433,13.736,5.0,2011.0,18,1.0
62,12.0,2010-12-17,1295605.35,0.0,52.77,3.236,126.879484,14.313,12.0,2010.0,50,3.0
68,12.0,2011-08-12,955913.68,0.0,91.04,3.701,129.201581,13.503,8.0,2011.0,32,2.0
93,12.0,2011-05-27,964332.51,0.0,NaN,4.087,129.062355,13.736,5.0,2011.0,21,4.0
135,12.0,2010-09-10,903119.03,1.0,83.63,3.044,126.114581,14.180,9.0,2010.0,36,2.0


In [22]:
df1[df1["Store"]==12]

,Store,Date,Weekly_Sales,Holiday_Flag,Temperature,Fuel_Price,CPI,Unemployment,Month,Year,Week_of_year,week
30,12.0,2011-05-06,1021154.48,0.0,68.40,4.193,129.044433,13.736,5.0,2011.0,18,1.0
62,12.0,2010-12-17,1295605.35,0.0,52.77,3.236,126.879484,14.313,12.0,2010.0,50,3.0
68,12.0,2011-08-12,955913.68,0.0,91.04,3.701,129.201581,13.503,8.0,2011.0,32,2.0
93,12.0,2011-05-27,964332.51,0.0,NaN,4.087,129.062355,13.736,5.0,2011.0,21,4.0
135,12.0,2010-09-10,903119.03,1.0,83.63,3.044,126.114581,14.180,9.0,2010.0,36,2.0


The unemployement rate looks particularly high in 5 entries. All of these entries being from the same store 12, we can suppose that store 12 is located in an area with higher unemployment rate. However, we will need to remove these outliers to prevent bias in the predictions. 

There is no outliers in the other columns.

In [23]:
columns_to_plot = ['Store', 'Holiday_Flag', 'Temperature',
                   'Fuel_Price', 'CPI', 'Unemployment',
                   'Month', 'Year','week']

# Create subplot grid: 3 columns per row
n_cols = 3
n_rows = -(-len(columns_to_plot) // n_cols)  # ceiling division

fig = make_subplots(rows=n_rows, cols=n_cols,
                    subplot_titles=[f"{col} vs Weekly_Sales" for col in columns_to_plot],
                    horizontal_spacing=0.1, vertical_spacing=0.15)

# Add scatter plots to the grid
for i, col in enumerate(columns_to_plot):
    row = i // n_cols + 1
    col_num = i % n_cols + 1
    fig.add_trace(
        go.Scatter(
            x=df1[col],
            y=df1['Weekly_Sales'],
            mode='markers',
            marker=dict(size=4, opacity=0.6),
            name=col
        ),
        row=row,
        col=col_num
    )

# Layout and appearance
fig.update_layout(
    height=300 * n_rows,
    width=1100,
    title_text="Bivariate Analysis: Weekly_Sales vs Other Variables",
    showlegend=False
)

fig.show()


In [24]:
df1.columns

Index(['Store', 'Date', 'Weekly_Sales', 'Holiday_Flag', 'Temperature',
       'Fuel_Price', 'CPI', 'Unemployment', 'Month', 'Year', 'Week_of_year',
       'week'],
      dtype='object')

In [25]:
df1.dtypes

Store                  float64
Date            datetime64[ns]
Weekly_Sales           float64
Holiday_Flag           float64
Temperature            float64
Fuel_Price             float64
CPI                    float64
Unemployment           float64
Month                  float64
Year                   float64
Week_of_year            UInt32
week                   float64
dtype: object

In [26]:
# Correlation matrix
corr_matrix = df1.corr(numeric_only=True).round(2)


fig = ff.create_annotated_heatmap(corr_matrix.values,
                                  x = corr_matrix.columns.tolist(),
                                  y = corr_matrix.index.tolist())


fig.show()

Average weekly sales are around $1.25M, with a median of approximately $1.25M as well. They range from $269K to $2.77M, indicating significant variation throughout the year.

The weekly sales are slightly higher at the begining and at the end of the month (week 1 and week 4).

Sales peak in December, averaging $2.15M per week, with the first week of the month reaching as high as $2.45M. January and February follow, with average weekly sales of around $1.52M.

A slight increase is observed in the summer months, rising from April ($934K per week) to July ($1.33M per week), followed by a decline from July to October ($812K per week).

From the data we have, stores 13 and 15 recorded the highest total weekly sales. However, store 4 had the highest average weekly sales.

Aside from that, no clear correlation is observed between weekly sales and other variables — except for a slight negative correlation with the CPI.

## Part 2 : Train a linera regression model

## Preprocessing

### 1. Dealing with missing values in Holiday_Flag 

In [27]:
# Check for inconsistencies in Holiday_Flag for each date. 
# If > 1, some similar dates have different Holiday_Flag values.
inconsistent_dates = df1.groupby('Date')['Holiday_Flag'].apply(lambda x: x.nunique() > 1)

# Filter dates with inconsistencies
inconsistent_dates = inconsistent_dates[inconsistent_dates].index

if inconsistent_dates.empty:
    print("All dates with filled Holiday_Flag values have the same value.")
else:
    print("The following dates have inconsistencies in Holiday_Flag:")
    print(inconsistent_dates)

All dates with filled Holiday_Flag values have the same value.


In [28]:
missing_holiday = df1[df1["Holiday_Flag"].isnull()]
missing_holiday

,Store,Date,Weekly_Sales,Holiday_Flag,Temperature,Fuel_Price,CPI,Unemployment,Month,Year,Week_of_year,week
0,6.0,2011-02-18,1572117.54,NaN,59.61,3.045,214.777523,6.858,2.0,2011.0,7,3.0
15,6.0,2010-04-30,1498080.16,NaN,68.91,2.780,211.894272,7.092,4.0,2010.0,17,5.0
43,7.0,2011-08-26,629994.47,NaN,57.60,3.485,194.379637,8.622,8.0,2011.0,34,4.0
48,1.0,2011-08-05,1624383.75,NaN,91.65,3.684,215.544618,7.962,8.0,2011.0,31,1.0
53,14.0,2011-03-25,1879451.23,NaN,41.76,3.625,184.994368,8.549,3.0,2011.0,12,4.0
65,10.0,NaT,1714309.90,NaN,43.43,3.287,127.191774,8.744,NaN,NaN,<NA>,NaN
73,1.0,2010-08-27,1449142.92,NaN,85.22,2.619,211.567306,7.787,8.0,2010.0,34,4.0
90,9.0,2010-07-09,485389.15,NaN,78.51,2.642,214.656430,6.442,7.0,2010.0,27,2.0
109,1.0,2011-12-09,NaN,NaN,43.93,3.158,218.961846,7.866,12.0,2011.0,49,2.0
118,9.0,2010-06-18,513073.87,NaN,82.99,2.637,215.016648,6.384,6.0,2010.0,24,3.0


In [29]:
# Create a dictionary mapping dates to Holiday_Flag values from no_missing_holiday
no_missing_holiday = df1.drop(missing_holiday.index)
no_missing_holiday=no_missing_holiday[["Date","Holiday_Flag"]]
no_missing_holiday=no_missing_holiday.drop_duplicates(subset='Date')
no_missing_holiday = no_missing_holiday.dropna(subset=['Date'])

holiday_map = no_missing_holiday.set_index('Date')['Holiday_Flag'].to_dict()


In [30]:
# Fill missing values in Holyday_Flag using the dictionary
df1['Holiday_Flag'] = df1.apply(
    lambda row: holiday_map.get(row['Date'], row['Holiday_Flag']), axis=1
)
df1.isnull().sum()

Store            0
Date            18
Weekly_Sales    14
Holiday_Flag     6
Temperature     18
Fuel_Price      14
CPI             12
Unemployment    15
Month           18
Year            18
Week_of_year    18
week            18
dtype: int64

In [31]:
# Fill missing values that have not been replaced in Holyday_Flag by 0
df1['Holiday_Flag'] = df1['Holiday_Flag'].fillna(0)
df1.isnull().sum()

Store            0
Date            18
Weekly_Sales    14
Holiday_Flag     0
Temperature     18
Fuel_Price      14
CPI             12
Unemployment    15
Month           18
Year            18
Week_of_year    18
week            18
dtype: int64

In [32]:
# Droping Date and Week_of_year column as they won't be needed anymore
df1 = df1.drop("Date", axis=1)
df1 = df1.drop("Week_of_year", axis=1)

In [33]:
df1.shape

(150, 10)

### 2. Dropping lines with missing value in the target and in date

In [34]:
# No imputation can be done to replace missing values on target variable since it could bias the model. 
# We will need to drop these lines.
df1 = df1.dropna(subset=['Weekly_Sales'])
df1.isnull().sum()

Store            0
Weekly_Sales     0
Holiday_Flag     0
Temperature     15
Fuel_Price      12
CPI             11
Unemployment    14
Month           18
Year            18
week            18
dtype: int64

In [35]:
# No logical imputation can be done to replace missing values on Date variable.
# We will need to drop these lines.
df1 = df1.dropna(subset=['Month'])
df1 = df1.dropna(subset=['Year'])
df1 = df1.dropna(subset=['week'])

df1.isnull().sum()

Store            0
Weekly_Sales     0
Holiday_Flag     0
Temperature     11
Fuel_Price      11
CPI              9
Unemployment    11
Month            0
Year             0
week             0
dtype: int64

In [36]:
df1.shape

(118, 10)

### 3. Dropping lines with outliers

In this project, will be considered as outliers all the numeric features that don't fall within the range : $[\bar{X} - 3\sigma, \bar{X} + 3\sigma]$. As said earlier, this concerns the columns *Unemployment*.

In [37]:
# Drop lines containing outliers without droping lines with missing values 
columns_to_clean = ['Unemployment']

for column in columns_to_clean:
    mean = df1[column].mean()
    std = df1[column].std()
    lower_bound = mean - 3 * std
    upper_bound = mean + 3 * std
    df1 = df1[(df1[column].isna()) | ((df1[column] > lower_bound) & (df1[column] < upper_bound))]
df1.shape

(113, 10)

In [38]:
df1.isnull().sum()

Store            0
Weekly_Sales     0
Holiday_Flag     0
Temperature     10
Fuel_Price      11
CPI              9
Unemployment    11
Month            0
Year             0
week             0
dtype: int64

### 4. Identifying the target variable and spliting in train and test sets

In [39]:
# Separate target variable Y from features X
target_name = "Weekly_Sales"

Y = df1.loc[:, target_name]
X = df1.drop(target_name, axis=1)

print(Y.head())
print()
print(X.head())
print()

0    1572117.54
1    1807545.43
4    1644470.66
5    1857533.70
6     695396.19
Name: Weekly_Sales, dtype: float64

   Store  Holiday_Flag  Temperature  Fuel_Price         CPI  Unemployment  \
0    6.0           0.0        59.61       3.045  214.777523         6.858   
1   13.0           0.0        42.38       3.435  128.616064         7.470   
4    6.0           0.0        78.89       2.759  212.412888         7.092   
5    4.0           0.0          NaN       2.756  126.160226         7.896   
6   15.0           0.0        69.80       4.069  134.855161         7.658   

   Month    Year  week  
0    2.0  2011.0   3.0  
1    3.0  2011.0   4.0  
4    5.0  2010.0   4.0  
5    5.0  2010.0   4.0  
6    6.0  2011.0   1.0  



In [40]:
# divide dataset into train set & test set 
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

### 5. Imputation of missing values, standardizing and one-hot encoding 

In [41]:
# identifying numeric and categoric features
numeric_features = ['Temperature', 'Fuel_Price','CPI', 'Unemployment']  
categoric_features = ['Store', 'Holiday_Flag','Month', 'Year', 'week']

In [43]:
missing_values = X_train.isnull().sum()
missing_values

Store           0
Holiday_Flag    0
Temperature     9
Fuel_Price      8
CPI             8
Unemployment    9
Month           0
Year            0
week            0
dtype: int64

In [44]:
missing_values = X_test.isnull().sum()
missing_values

Store           0
Holiday_Flag    0
Temperature     1
Fuel_Price      3
CPI             1
Unemployment    2
Month           0
Year            0
week            0
dtype: int64

In [45]:
preprocessor = ColumnTransformer([
  ('num', Pipeline([
    ('num_imputer', KNNImputer()), # Imputing with KNN since the value can correlated to the Date or to the Store
    ('num_scaler', StandardScaler())
  ]), numeric_features),
  ('cat', Pipeline([
    ('cat_encoder', OneHotEncoder(drop="if_binary")) # No more missing values in these variables
  ]), categoric_features)
])

preprocessor.fit(X_train)

X_train = preprocessor.transform(X_train)
X_test = preprocessor.transform(X_test)

## Base line model

In [46]:
baseline_model = LinearRegression()
baseline_model.fit(X_train, Y_train)
print(f"R2 Train: {baseline_model.score(X_train, Y_train)}\nR2 Test: {baseline_model.score(X_test, Y_test)}")

# Predictions on train and test
Y_train_pred = baseline_model.predict(X_train)
Y_test_pred = baseline_model.predict(X_test)

# root_mean_squared_error
print("RMSE Train : ", root_mean_squared_error(Y_train, Y_train_pred))
print("RMSE Test : ", root_mean_squared_error(Y_test, Y_test_pred))

R2 Train: 0.991766661376624
R2 Test: 0.937586008115087
RMSE Train :  62813.05498097165
RMSE Test :  145562.5802796255


First, we notice a gap between the R² train score and the R² test score, with the train score being 0.06 points higher, which suggests some overfitting.

Second, the root mean square error is particularly high for both sets, meaning our model is, on average, off by $63K on the training set and $146K on the test set.

Since the dataset only has 113 lines, let's do a cross validation to confirm the overfitting hypothesis.

In [47]:
cv_scores = cross_val_score(baseline_model,X_train, Y_train, cv=5)
avg = cv_scores.mean()
std = cv_scores.std()
print("R2 Train :", baseline_model.score(X_train, Y_train))
print(f'Cross-validated R2 score : {avg}')
print(f'Standard deviation : {std}')
print("R2 score on test set can vary from", baseline_model.score(X_test, Y_test) - std, "to", baseline_model.score(X_test, Y_test) + std, "\nthat is smaller than the cross-validated score")


R2 Train : 0.991766661376624
Cross-validated R2 score : 0.9302904178779748
Standard deviation : 0.039914361608993254
R2 score on test set can vary from 0.8976716465060937 to 0.9775003697240803 
that is smaller than the cross-validated score


These results confirm some degree of overfitting. The R² score on the training set (0.991) is significantly higher than the cross-validated score (0.93). Even if we take in consideration the standard deviation, the R² score on test set remains smaller. The overfitting hypothesis is confirmed and indicate that the model may not generalize well to unseen data.

In [48]:
column_names = []
for name, pipeline, features_list in preprocessor.transformers_: # loop over pipelines
    if name == 'num': # if pipeline is for numeric variables
        features = features_list # just get the names of columns to which it has been applied
    else: # for categorical variables
        features = pipeline.named_steps['cat_encoder'].get_feature_names_out() # get output columns names from OneHotEncoder
    column_names.extend(features) # concatenate features names
        
print("Names of columns corresponding to each coefficient: ", column_names)

Names of columns corresponding to each coefficient:  ['Temperature', 'Fuel_Price', 'CPI', 'Unemployment', 'Store_1.0', 'Store_2.0', 'Store_3.0', 'Store_4.0', 'Store_5.0', 'Store_6.0', 'Store_7.0', 'Store_8.0', 'Store_9.0', 'Store_10.0', 'Store_11.0', 'Store_13.0', 'Store_14.0', 'Store_15.0', 'Store_16.0', 'Store_17.0', 'Store_18.0', 'Store_19.0', 'Store_20.0', 'Holiday_Flag_1.0', 'Month_1.0', 'Month_2.0', 'Month_3.0', 'Month_4.0', 'Month_5.0', 'Month_6.0', 'Month_7.0', 'Month_8.0', 'Month_9.0', 'Month_10.0', 'Month_11.0', 'Month_12.0', 'Year_2010.0', 'Year_2011.0', 'Year_2012.0', 'week_1.0', 'week_2.0', 'week_3.0', 'week_4.0', 'week_5.0']


In [49]:
coefs = pd.DataFrame(index = column_names, data = baseline_model.coef_.transpose(), columns=["coefficients"])

In [50]:
coef = coefs.assign(abs_coefficients=coefs['coefficients'].abs()).sort_values(by='abs_coefficients', ascending=False).reset_index()
coef = coef.rename(columns={'index': 'categories'})
coef

,categories,coefficients,abs_coefficients
0,Store_5.0,-943908.790177,943908.790177
1,Store_3.0,-902027.652517,902027.652517
2,Store_14.0,849339.310616,849339.310616
3,Store_9.0,-805007.652384,805007.652384
4,Store_15.0,-753527.221914,753527.221914
5,Store_2.0,696635.591019,696635.591019
6,Store_16.0,-682903.817576,682903.817576
7,Store_11.0,658631.572570,658631.572570
8,Store_20.0,647037.176285,647037.176285
9,Store_13.0,635953.728883,635953.728883


In [51]:
fig = px.bar(coef, x="categories", y="coefficients", 
             labels={'categories': 'Categories', 'coefficients': 'Coefficients'}, 
             title='Coefficients per Category')
fig.show()

The store number and the month of december have a high impact on weekly sales. Indeed, as seen in EDA, store 5 and 3 perform less and store 14 performs better, impacting significantly the weekly_sales results.

Also, we can notice that many variable only have a very small impact on the target.

# Part 3 : Regularisation